## Import libraries


In [2]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType,StructField, IntegerType,StringType



## Create SparkSession object

In [3]:
builder = (SparkSession.builder
           .appName("connect-kafka-streaming")
           .master("spark://spark-master:7077")
           .config("spark.executor.memory", "512m")
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
          )

spark = configure_spark_with_delta_pip(builder, ['org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1']).getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
        

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-88552523-9fd8-4e7b-8545-da541b510812;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.1 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in centra

## Create a dataframe

In [4]:
df = (spark.readStream
      .format("kafka")
      .option("kafka.bootstrap.servers", "kafka:9092")
      .option("subscribe","users")
      .option("startingOffsets", "earliest")
      .load())

In [5]:
schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('age', IntegerType(), True),
    StructField('gender', StringType(), True),
    StructField('country', StringType(), True)])

df = df.withColumn('value', from_json(col('value').cast("STRING"), schema))

In [6]:
df = df.select(
    col('value.id').alias('id'),
    col('value.name').alias('name'),
    col('value.age').alias('age'),
    col('value.gender').alias('gender'),
    col('value.country').alias('country'))

In [7]:
query = (df.writeStream
    .outputMode('append')
    .format('console')
    .start())

-------------------------------------------
Batch: 0
-------------------------------------------
+---+-------+---+------+---------+
| id|   name|age|gender|  country|
+---+-------+---+------+---------+
| 94| user59| 36|     F|    Chine|
| 83| user33| 23|     F|    India|
|  1| user53| 63|     M|    India|
| 14| user57| 27|     M|   Brazil|
| 15| user80| 31|     F|Australia|
| 69|  user4| 37|     M|       UK|
| 30| user93| 42|     F|   Brazil|
| 10| user13| 34|     F|   Canada|
| 77| user89| 22|     F|   Brazil|
| 78| user89| 19|     M|   Brazil|
| 96| user46| 33|     F|       UK|
| 81| user67| 21|     F|   Brazil|
| 28| user77| 33|     F|    India|
| 88| user12| 24|     M|    Chine|
|  1| user89| 24|     M|Australia|
| 70|user100| 49|     F|    India|
| 15| user48| 38|     M|    India|
| 45| user30| 43|     F|   Brazil|
| 32| user97| 27|     F|   Canada|
| 47| user74| 22|     F|      USA|
+---+-------+---+------+---------+
only showing top 20 rows



-------------------------------------------
Batch: 1
-------------------------------------------
+---+------+---+------+-------+
| id|  name|age|gender|country|
+---+------+---+------+-------+
|  5|user83| 37|     F|    USA|
+---+------+---+------+-------+

-------------------------------------------
Batch: 2
-------------------------------------------
+---+------+---+------+-------+
| id|  name|age|gender|country|
+---+------+---+------+-------+
| 93|user37| 51|     F|    USA|
+---+------+---+------+-------+



-------------------------------------------
Batch: 3
-------------------------------------------
+---+-----+---+------+-------+
| id| name|age|gender|country|
+---+-----+---+------+-------+
| 39|user2| 44|     F|  India|
+---+-----+---+------+-------+



In [8]:
query.stop()

In [9]:
spark.stop()